In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import PyPDF2 
import tabula
from io import StringIO
from csv import reader
from urllib.request import urlopen
import re
from IPython.core.display import HTML

%matplotlib inline

# Webscraping to find list of Winter Games

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
url_list = 'https://olympics.com/en/olympic-games/olympic-results'
r = requests.get(url_list, headers=headers)
soup = bs(r.text, 'html.parser')
games = soup.find_all('section', {'class':'styles__ItemLink-sc-111k07d-2 ipqQzr link-item'})

list_of_games = []
for item in games:
    olympic_games = item.find('p').text
    season =  item.find('span').text
    list_of_games.append((olympic_games, season))


In [3]:
list_of_games = pd.DataFrame({'city':list_of_games})
print(list_of_games)

                                      city
0                   (Beijing 2022, Winter)
1                     (Tokyo 2020, Summer)
2               (PyeongChang 2018, Winter)
3                       (Rio 2016, Summer)
4                     (Sochi 2014, Winter)
..                                     ...
101              (St. Moritz 1948, Winter)
102  (Garmisch-Partenkirchen 1936, Winter)
103             (Lake Placid 1932, Winter)
104              (St. Moritz 1928, Winter)
105                (Chamonix 1924, Winter)

[106 rows x 1 columns]


In [4]:
list_of_games['city'] = list_of_games['city'].astype('string')
winter_games = list_of_games[list_of_games['city'].str.contains('Winter')]
print(winter_games)

                                          city
0                   ('Beijing 2022', 'Winter')
2               ('PyeongChang 2018', 'Winter')
4                     ('Sochi 2014', 'Winter')
6                 ('Vancouver 2010', 'Winter')
8                     ('Turin 2006', 'Winter')
10           ('Salt Lake City 2002', 'Winter')
12                   ('Nagano 1998', 'Winter')
14              ('Lillehammer 1994', 'Winter')
16              ('Albertville 1992', 'Winter')
18                  ('Calgary 1988', 'Winter')
20                 ('Sarajevo 1984', 'Winter')
22              ('Lake Placid 1980', 'Winter')
24                ('Innsbruck 1976', 'Winter')
26                  ('Sapporo 1972', 'Winter')
28                 ('Grenoble 1968', 'Winter')
30                ('Innsbruck 1964', 'Winter')
32             ('Squaw Valley 1960', 'Winter')
34        ("Cortina d'Ampezzo 1956", 'Winter')
36                     ('Oslo 1952', 'Winter')
38               ('St. Moritz 1948', 'Winter')
40   ('Garmis

In [5]:
winter_games = winter_games.replace(to_replace='\(\'', value="", regex=True)
winter_games = winter_games.replace(to_replace=r'\'\)', value="", regex=True)
winter_games = winter_games.replace(to_replace=r'\)', value="", regex=True)
winter_games = winter_games.replace(to_replace=r'\'',value="",regex=True)
winter_games = winter_games.replace(to_replace=r'\,',value="",regex=True)
winter_games = winter_games.replace(to_replace='\(',value="",regex=True)
winter_games = winter_games.replace(to_replace='\"',value="",regex=True)
winter_games

,city
0,Beijing 2022 Winter
2,PyeongChang 2018 Winter
4,Sochi 2014 Winter
6,Vancouver 2010 Winter
8,Turin 2006 Winter
10,Salt Lake City 2002 Winter
12,Nagano 1998 Winter
14,Lillehammer 1994 Winter
16,Albertville 1992 Winter
18,Calgary 1988 Winter


In [6]:
winter_games['season'] = winter_games['city'].apply(lambda x: x.split(' ')[-1])
winter_games['city'] = winter_games.city.str.rsplit(' ',1).str[0]
winter_games

,city,season
0,Beijing 2022,Winter
2,PyeongChang 2018,Winter
4,Sochi 2014,Winter
6,Vancouver 2010,Winter
8,Turin 2006,Winter
10,Salt Lake City 2002,Winter
12,Nagano 1998,Winter
14,Lillehammer 1994,Winter
16,Albertville 1992,Winter
18,Calgary 1988,Winter


In [7]:
winter_games['year'] = winter_games['city'].apply(lambda x: x.split(' ')[-1])
winter_games['city'] = winter_games.city.str.rsplit(' ',1).str[0]
winter_games

,city,season,year
0,Beijing,Winter,2022
2,PyeongChang,Winter,2018
4,Sochi,Winter,2014
6,Vancouver,Winter,2010
8,Turin,Winter,2006
10,Salt Lake City,Winter,2002
12,Nagano,Winter,1998
14,Lillehammer,Winter,1994
16,Albertville,Winter,1992
18,Calgary,Winter,1988


In [8]:
winter_games = winter_games.reset_index()
winter_games

,index,city,season,year
0,0,Beijing,Winter,2022
1,2,PyeongChang,Winter,2018
2,4,Sochi,Winter,2014
3,6,Vancouver,Winter,2010
4,8,Turin,Winter,2006
5,10,Salt Lake City,Winter,2002
6,12,Nagano,Winter,1998
7,14,Lillehammer,Winter,1994
8,16,Albertville,Winter,1992
9,18,Calgary,Winter,1988


In [9]:
# winter_games.drop('index', axis=1, inplace=True)
# code to drop index column is giving error 

In [10]:
winter_games['city'] = winter_games['city'] + '-'
winter_games

,index,city,season,year
0,0,Beijing-,Winter,2022
1,2,PyeongChang-,Winter,2018
2,4,Sochi-,Winter,2014
3,6,Vancouver-,Winter,2010
4,8,Turin-,Winter,2006
5,10,Salt Lake City-,Winter,2002
6,12,Nagano-,Winter,1998
7,14,Lillehammer-,Winter,1994
8,16,Albertville-,Winter,1992
9,18,Calgary-,Winter,1988


In [11]:
winter_games['city_year'] = winter_games['city'] + winter_games['year']
winter_games['city_year'] = winter_games['city_year'].str.lower()
winter_games

,index,city,season,year,city_year
0,0,Beijing-,Winter,2022,beijing-2022
1,2,PyeongChang-,Winter,2018,pyeongchang-2018
2,4,Sochi-,Winter,2014,sochi-2014
3,6,Vancouver-,Winter,2010,vancouver-2010
4,8,Turin-,Winter,2006,turin-2006
5,10,Salt Lake City-,Winter,2002,salt lake city-2002
6,12,Nagano-,Winter,1998,nagano-1998
7,14,Lillehammer-,Winter,1994,lillehammer-1994
8,16,Albertville-,Winter,1992,albertville-1992
9,18,Calgary-,Winter,1988,calgary-1988


In [12]:
winter_games['city_year'] = winter_games['city_year'].str.replace(' ','-').str.replace('.','')

C:\Users\dkham\AppData\Local\Temp/ipykernel_22272/3568862458.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  winter_games['city_year'] = winter_games['city_year'].str.replace(' ','-').str.replace('.','')


In [13]:
winter_games_list = winter_games['city_year'].tolist()

In [14]:
winter_games_list

['beijing-2022',
 'pyeongchang-2018',
 'sochi-2014',
 'vancouver-2010',
 'turin-2006',
 'salt-lake-city-2002',
 'nagano-1998',
 'lillehammer-1994',
 'albertville-1992',
 'calgary-1988',
 'sarajevo-1984',
 'lake-placid-1980',
 'innsbruck-1976',
 'sapporo-1972',
 'grenoble-1968',
 'innsbruck-1964',
 'squaw-valley-1960',
 'cortina-dampezzo-1956',
 'oslo-1952',
 'st-moritz-1948',
 'garmisch-partenkirchen-1936',
 'lake-placid-1932',
 'st-moritz-1928',
 'chamonix-1924',
 'beijing-2022',
 'pyeongchang-2018',
 'sochi-2014',
 'vancouver-2010',
 'turin-2006',
 'salt-lake-city-2002',
 'nagano-1998',
 'lillehammer-1994',
 'albertville-1992',
 'calgary-1988',
 'sarajevo-1984',
 'lake-placid-1980',
 'innsbruck-1976',
 'sapporo-1972',
 'grenoble-1968',
 'innsbruck-1964',
 'squaw-valley-1960',
 'cortina-dampezzo-1956',
 'oslo-1952',
 'st-moritz-1948',
 'garmisch-partenkirchen-1936',
 'lake-placid-1932',
 'st-moritz-1928',
 'chamonix-1924']

# Women's Analysis

In [15]:
# women's individual medalist by game 

w_names_list = []
w_medals_list = []
w_countries_list = []
w_winter_games_list =[]

for winter_games in winter_games_list:
    '''scraping through olympic url to find women's figure skating results'''
    if winter_games == 'pyeongchang-2018':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/ladies-single-skating'
    elif winter_games == 'beijing-2022':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/women-single-skating'
    else:
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/individual-women'
    
    res = requests.get(url)
    soup = bs(res.content)
    
    for row in soup.findAll('div', {'data-cy':'single-athlete-result-row'})[:3]:
        '''pulling the women's names, medals, countries from olympics url'''
        name = row.h3.text
        medal = row.span.text
        country = row.findAll('span',{'data-cy':True})[1].text

        w_names_list.append(name)
        w_medals_list.append(medal)
        w_countries_list.append(country)
        w_winter_games_list.append(winter_games)

In [16]:
games_women_dict = {
    'name': w_names_list,
    'medal': w_medals_list,
    'country': w_countries_list,
    'games': w_winter_games_list
}

w_medals = pd.DataFrame(games_women_dict)
w_medals['year'] = w_medals['games'].apply(lambda x: x.split('-')[-1])
w_medals

#create dictionary, keys, run .map on country column, df['col'].map(dict)

,name,medal,country,games,year
0,anna shcherbakova,G,ROC,beijing-2022,2022
1,alexandra trusova,S,ROC,beijing-2022,2022
2,kaori sakamoto,B,JPN,beijing-2022,2022
3,alina zagitova,G,OAR,pyeongchang-2018,2018
4,evgenia medvedeva,S,OAR,pyeongchang-2018,2018
...,...,...,...,...,...
133,fritzi burger,S,AUT,st-moritz-1928,1928
134,beatrix loughran,B,USA,st-moritz-1928,1928
135,herma planck-szabo,G,AUT,chamonix-1924,1924
136,beatrix loughran,S,USA,chamonix-1924,1924


### Women's Medals by Age

In [17]:
medals_by_age_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_Olympic_medalists_in_figure_skating_by_age')
w_medals_by_age = medals_by_age_df[2]
w_medals_by_age

,Rank,Age,Nation & Name,Games,Medal,Date of Birth,Date of Event,Notes
0,1,"15 years, 79 days",Cecilia Colledge,1936 Garmisch-Partenkirchen,Silver,"November 28, 1920","February 15, 1936",NaN
1,2,"15 years, 249 days",Yulia Lipnitskaya,2014 Sochi,Gold,"June 5, 1998","February 9, 2014",In team event
2,3,"15 years, 255 days",Tara Lipinski,1998 Nagano,Gold,"June 10, 1982","February 20, 1998",NaN
3,4,"15 years, 261 days",Ingrid Wendl,1956 Cortina d'Ampezzo,Bronze,"May 17, 1940","February 2, 1956",NaN
4,5,"15 years, 270 days",Alina Zagitova,2018 Pyeongchang,Silver,"May 18, 2002","February 12, 2018",In team event; her 1st medal
...,...,...,...,...,...,...,...,...
87,88,"27 years, 12 days",Carolina Kostner,2014 Sochi,Bronze,"February 8, 1987","February 20, 2014",NaN
88,89,"27 years, 14 days",Irina Slutskaya,2006 Torino,Bronze,"February 9, 1979","February 23, 2006",Her 2nd medal
89,90,"27 years, 43 days",Madge Syers,1908 London,Gold,"September 16, 1881","October 29, 1908",Her 1st medal [c]
90,91,"27 years, 233 days",Beatrix Loughran,1928 St. Moritz,Bronze,"June 30, 1900","February 18, 1928",Her 2nd medal [b]


In [18]:
w_medals_by_age = w_medals_by_age.rename(columns={'Nation & Name':'Name'})
w_medals_by_age.columns = [
    col.strip().lower().replace(' ','_')
    for col in w_medals_by_age
]
w_medals_by_age['name'] = w_medals_by_age['name'].str.lower()
w_medals_by_age = w_medals_by_age[~w_medals_by_age.notes.fillna('') \
                                  .str.lower() \
                                  .str.contains("team")
                                   ]
w_medals_by_age['year'] = w_medals_by_age['date_of_event'].apply(lambda x: x.split(' ')[-1])
w_medals_by_age

,rank,age,name,games,medal,date_of_birth,date_of_event,notes,year
0,1,"15 years, 79 days",cecilia colledge,1936 Garmisch-Partenkirchen,Silver,"November 28, 1920","February 15, 1936",NaN,1936
2,3,"15 years, 255 days",tara lipinski,1998 Nagano,Gold,"June 10, 1982","February 20, 1998",NaN,1998
3,4,"15 years, 261 days",ingrid wendl,1956 Cortina d'Ampezzo,Bronze,"May 17, 1940","February 2, 1956",NaN,1956
5,6,"15 years, 281 days",alina zagitova,2018 Pyeongchang,Gold,"May 18, 2002","February 23, 2018",Her 2nd medal,2018
7,8,"15 years, 316 days",sonja henie,1928 St. Moritz,Gold,"April 8, 1912","February 18, 1928",Her 1st medal,1928
...,...,...,...,...,...,...,...,...,...
87,88,"27 years, 12 days",carolina kostner,2014 Sochi,Bronze,"February 8, 1987","February 20, 2014",NaN,2014
88,89,"27 years, 14 days",irina slutskaya,2006 Torino,Bronze,"February 9, 1979","February 23, 2006",Her 2nd medal,2006
89,90,"27 years, 43 days",madge syers,1908 London,Gold,"September 16, 1881","October 29, 1908",Her 1st medal [c],1908
90,91,"27 years, 233 days",beatrix loughran,1928 St. Moritz,Bronze,"June 30, 1900","February 18, 1928",Her 2nd medal [b],1928


In [81]:
# merging medals DF and age DF
w_medals_merged = (
    w_medals.merge(
    w_medals_by_age[['name','age','medal','date_of_birth','year']], #df2 merged
    on = ['name','year'], 
    how ='left'
    ))
#adding column for age in just year value
w_medals_merged['age'] = w_medals_merged['age'].astype('string')
w_medals_merged['age_years'] = w_medals_merged['age'].fillna('').str.split(' ').str[0]
w_medals_merged['age_years'] = w_medals_merged['age_years'].astype('string')
w_medals_merged['age_years'] = w_medals_merged.age_years.str.replace('years,','')

#combining ROC and OAR with RUS
w_medals_merged['country'] = w_medals_merged['country'].replace({'ROC':'RUS', 'OAR':'RUS'})
w_medals_merged

# unsuccessful strip formulas for removing 'years,' from age
# w_medals_merged['age_years'] = w_medals_merged['age_years'].map(lambda x: x.rstrip(' years,')
# w_medals_merged['age_years'] = [x.rstrip('years,') for x in w_medals_merged['age_years']]

,name,medal_x,country,games,year,age,medal_y,date_of_birth,age_years
0,anna shcherbakova,G,RUS,beijing-2022,2022,"17 years, 326 days",Gold,"March 28, 2004",17
1,alexandra trusova,S,RUS,beijing-2022,2022,"17 years, 239 days",Silver,"June 23, 2004",17
2,kaori sakamoto,B,JPN,beijing-2022,2022,"21 years, 314 days",Bronze,"April 9, 2000",21
3,alina zagitova,G,RUS,pyeongchang-2018,2018,"15 years, 281 days",Gold,"May 18, 2002",15
4,evgenia medvedeva,S,RUS,pyeongchang-2018,2018,"18 years, 96 days",Silver,"November 19, 1999",18
...,...,...,...,...,...,...,...,...,...
133,fritzi burger,S,AUT,st-moritz-1928,1928,"17 years, 257 days",Silver,"June 6, 1910",17
134,beatrix loughran,B,USA,st-moritz-1928,1928,"27 years, 233 days",Bronze,"June 30, 1900",27
135,herma planck-szabo,G,AUT,chamonix-1924,1924,<NA>,NaN,NaN,
136,beatrix loughran,S,USA,chamonix-1924,1924,"23 years, 215 days",Silver,"June 30, 1900",23


In [82]:
w_medals_merged.to_excel('w_medals.xlsx')

In [83]:
w_medals_merged.groupby(['year','country']).count()

name  medal_x  games  age  medal_y  date_of_birth  age_years
year country                                                              
1924 AUT         2        2      2    0        0              0          2
     GBR         2        2      2    2        2              2          2
     USA         2        2      2    2        2              2          2
1928 AUT         2        2      2    2        2              2          2
     NOR         2        2      2    2        2              2          2
...            ...      ...    ...  ...      ...            ...        ...
2014 RUS         2        2      2    2        2              2          2
2018 CAN         2        2      2    2        2              2          2
     RUS         4        4      4    4        4              4          4
2022 JPN         2        2      2    2        2              2          2
     RUS         4        4      4    4        4              4          4

[63 rows x 7 columns]

In [76]:
w_medals_merged.sort_values('age')

,name,country,games,year,age,medal,date_of_birth,age_years
87,tara lipinski,USA,nagano-1998,1998,"15 years, 255 days",Gold,"June 10, 1982",15
18,tara lipinski,USA,nagano-1998,1998,"15 years, 255 days",Gold,"June 10, 1982",15
3,alina zagitova,RUS,pyeongchang-2018,2018,"15 years, 281 days",Gold,"May 18, 2002",15
72,alina zagitova,RUS,pyeongchang-2018,2018,"15 years, 281 days",Gold,"May 18, 2002",15
132,sonja henie,NOR,st-moritz-1928,1928,"15 years, 316 days",Gold,"April 8, 1912",15
...,...,...,...,...,...,...,...,...
108,beatrix schuba,AUT,sapporo-1972,1972,<NA>,NaN,NaN,
113,hana maskova,TCH,grenoble-1968,1968,<NA>,NaN,NaN,
123,barbara ann scott,CAN,st-moritz-1948,1948,<NA>,NaN,NaN,
128,vivi-anne hulten,SWE,garmisch-partenkirchen-1936,1936,<NA>,NaN,NaN,


# Men's Analysis

In [20]:
# men's individual medalist 

m_names_list = []
m_medals_list = []
m_countries_list = []
m_winter_games_list =[]

for winter_games in winter_games_list:
    '''scraping through olympic url to find men's figure skating results'''
    if winter_games == 'pyeongchang-2018':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/mens-single-skating'
    elif winter_games == 'beijing-2022':
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/men-single-skating'
    else:
        url = f'https://olympics.com/en/olympic-games/{winter_games}/results/figure-skating/individual-men'
    
    res = requests.get(url)
    soup = bs(res.content)
    
    for row in soup.findAll('div', {'data-cy':'single-athlete-result-row'})[:3]:
        '''pulling the men's names, medals, countries from olympics url'''
        name = row.h3.text
        medal = row.span.text
        country = row.findAll('span',{'data-cy':True})[1].text

        m_names_list.append(name)
        m_medals_list.append(medal)
        m_countries_list.append(country)
        m_winter_games_list.append(winter_games)

### Men's Medals by Game

In [21]:
games_men_dict = {
    'name': m_names_list,
    'medal': m_medals_list,
    'country': m_countries_list,
    'games':m_winter_games_list
}

m_medals = pd.DataFrame(games_men_dict)
m_medals['year'] = m_medals['games'].apply(lambda x: x.split('-')[-1])
m_medals

,name,medal,country,games,year
0,nathan chen,G,USA,beijing-2022,2022
1,yuma kagiyama,S,JPN,beijing-2022,2022
2,shoma uno,B,JPN,beijing-2022,2022
3,yuzuru hanyu,G,JPN,pyeongchang-2018,2018
4,shoma uno,S,JPN,pyeongchang-2018,2018
...,...,...,...,...,...
133,willy böckl,S,AUT,st-moritz-1928,1928
134,robert van zeebroeck,B,BEL,st-moritz-1928,1928
135,gillis grafström,G,SWE,chamonix-1924,1924
136,willy böckl,S,AUT,chamonix-1924,1924


### Men's Medals by Age

In [22]:
#creating men's medals by age DF by referencing first index of medals by age DF 
m_medals_by_age = medals_by_age_df[0]
m_medals_by_age

,Rank,Age,Nation & Name,Games,Medal,Date of Birth,Date of Event,Notes
0,1,"14 years, 363 days",Scott Allen,1964 Innsbruck,Bronze,"February 8, 1949","February 6, 1964",NaN
1,2,"18 years, 107 days",Robert Van Zeebroeck,1928 St. Moritz,Bronze,"October 31, 1909","February 15, 1928",NaN
2,3,"18 years, 129 days",Ronald Robertson,1956 Cortina d'Ampezzo,Silver,"September 25, 1937","February 1, 1956",NaN
3,4,"18 years, 157 days",Mark Kondratiuk,2022 Beijing,Gold,"September 3, 2003","February 7, 2022",In team event
4,5,"18 years, 202 days",Dick Button,1948 St Moritz,Gold,"July 18, 1929","February 5, 1948",His 1st medal
...,...,...,...,...,...,...,...,...
87,88,"31 years, 98 days",Evgeni Plushenko,2014 Sochi,Gold,"November 3, 1982","February 9, 2014",In team event; his 4th medal
88,89,"34 years, 253 days",Gillis Grafström,1928 St. Moritz,Gold,"June 7, 1893","February 15, 1928",His 3rd medal
89,90,"35 years, 19 days",Willy Böckl,1928 St. Moritz,Silver,"January 27, 1893","February 15, 1928",His 2nd medal
90,91,"38 years, 247 days",Gillis Grafström,1932 Lake Placid,Silver,"June 7, 1893","February 9, 1932",His 4th medal


In [23]:
# cleaning men's medals by age data
m_medals_by_age = m_medals_by_age.rename(columns={'Nation & Name':'Name'}) # renaming column name
m_medals_by_age.columns = [ 
    col.strip().lower().replace(' ','_')
    for col in m_medals_by_age
] # making column names lowercase
m_medals_by_age['name'] = m_medals_by_age['name'].str.lower() # making all names in column lowercase
m_medals_by_age = m_medals_by_age[~m_medals_by_age.notes.fillna('') \
                                  .str.lower() \
                                  .str.contains("team")
                                   ] # removing rows with "team" in notes, only want individual medal info
m_medals_by_age['year'] = m_medals_by_age['date_of_event'].apply(lambda x: x.split(' ')[-1])
m_medals_by_age

,rank,age,name,games,medal,date_of_birth,date_of_event,notes,year
0,1,"14 years, 363 days",scott allen,1964 Innsbruck,Bronze,"February 8, 1949","February 6, 1964",NaN,1964
1,2,"18 years, 107 days",robert van zeebroeck,1928 St. Moritz,Bronze,"October 31, 1909","February 15, 1928",NaN,1928
2,3,"18 years, 129 days",ronald robertson,1956 Cortina d'Ampezzo,Silver,"September 25, 1937","February 1, 1956",NaN,1956
4,5,"18 years, 202 days",dick button,1948 St Moritz,Gold,"July 18, 1929","February 5, 1948",His 1st medal,1948
5,6,"18 years, 238 days",viktor petrenko,1988 Calgary,Bronze,"June 27, 1969","February 20, 1988",His 1st medal,1988
...,...,...,...,...,...,...,...,...,...
86,87,"31 years, 83 days",ulrich salchow,1908 London,Gold,"August 7, 1877","October 29, 1908",NaN,1908
88,89,"34 years, 253 days",gillis grafström,1928 St. Moritz,Gold,"June 7, 1893","February 15, 1928",His 3rd medal,1928
89,90,"35 years, 19 days",willy böckl,1928 St. Moritz,Silver,"January 27, 1893","February 15, 1928",His 2nd medal,1928
90,91,"38 years, 247 days",gillis grafström,1932 Lake Placid,Silver,"June 7, 1893","February 9, 1932",His 4th medal,1932


In [84]:
# merging medals df with age df
m_medals_merged = (
    m_medals.merge(
    m_medals_by_age[['name','age','medal','date_of_birth','year']], #df2 merged
    on = ['name','year'], 
    how ='left'
    ))
#separating out age by age in years only
m_medals_merged['age'] = m_medals_merged['age'].astype('string')
m_medals_merged['age_years'] = m_medals_merged['age'].fillna('').str.split(' ').str[0]
m_medals_merged['age_years'] = m_medals_merged['age_years'].astype('string')
m_medals_merged['age_years'] = m_medals_merged.age_years.str.replace('years,','')
m_medals_merged

m_medals_merged

,name,medal_x,country,games,year,age,medal_y,date_of_birth,age_years
0,nathan chen,G,USA,beijing-2022,2022,"22 years, 281 days",Gold,"May 5, 1999",22
1,yuma kagiyama,S,JPN,beijing-2022,2022,"18 years, 281 days",Silver,"May 5, 2003",18
2,shoma uno,B,JPN,beijing-2022,2022,"24 years, 55 days",Bronze,"December 17, 1997",24
3,yuzuru hanyu,G,JPN,pyeongchang-2018,2018,"23 years, 72 days",Gold,"December 7, 1994",23
4,shoma uno,S,JPN,pyeongchang-2018,2018,"20 years, 62 days",Silver,"December 17, 1997",20
...,...,...,...,...,...,...,...,...,...
133,willy böckl,S,AUT,st-moritz-1928,1928,"35 years, 19 days",Silver,"January 27, 1893",35
134,robert van zeebroeck,B,BEL,st-moritz-1928,1928,"18 years, 107 days",Bronze,"October 31, 1909",18
135,gillis grafström,G,SWE,chamonix-1924,1924,"30 years, 237 days",Gold,"June 7, 1893",30
136,willy böckl,S,AUT,chamonix-1924,1924,"31 years, 3 days",Silver,"January 27, 1893",31


In [85]:
m_medals_merged.to_excel('m_medals.xlsx')